In [8]:
print("--- VERIFICATION FINALE DE COMPATIBILITÉ (CORRIGÉE) ---")

import xgboost as xgb
import os
import torch
import joblib
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2
from torchvision import models # ON UTILISE TORCHVISION COMME A L'ENTRAINEMENT

# config
device = torch.device("cuda")
path_data = r"C:\Users\amisf\Desktop\datascientest_projet"
path_out = os.path.join(path_data, "implementation", "outputs")

# 1. Chargement Data et Encodeur
print("Chargement de l'encodeur (Vérifie la date du fichier !)...")
path_enc = os.path.join(path_out, "M2_IMAGE_XGBoost_Encoder.pkl")
path_mod = os.path.join(path_out, "M2_IMAGE_Classic_XGBoost.json")

# Vérification timestamp pour être sur qu'on teste le nouveau
t_mod = os.path.getmtime(path_mod)
print(f"Dernière modification du modèle : {pd.to_datetime(t_mod, unit='s')}")

le_xgb = joblib.load(path_enc)

# Dataframe validation
df = pd.read_csv(os.path.join(path_data, "data", "raw", "X_train_update.csv"))
y = pd.read_csv(os.path.join(path_data, "data", "raw", "Y_train_CVw08PX.csv"))
if 'prdtypecode' not in y.columns: y = y.rename(columns={y.columns[1]: 'prdtypecode'})
df = df.merge(y, left_index=True, right_index=True)

path_img = os.path.join(path_data, "data", "raw", "images", "images", "image_train")
df['path'] = df.apply(lambda r: os.path.join(path_img, f"image_{r['imageid']}_product_{r['productid']}.jpg"), axis=1)
df = df[df['path'].apply(os.path.exists)]

# On prend 200 images, MAIS on s'assure de les encoder comme le modèle l'attend
_, df_val_mini = train_test_split(df, test_size=200, stratify=df['label'] if 'label' in df else None, random_state=42)
y_true = le_xgb.transform(df_val_mini['prdtypecode']) # Utilisation de l'encodeur chargé

# 2. EXTRACTION FEATURES (VERSION TORCHVISION STRICTE)
print("Extraction Features (Mode Torchvision)...")
# On reproduit EXACTEMENT l'architecture de l'entrainement
resnet = models.resnet50(weights="IMAGENET1K_V1")
extractor = torch.nn.Sequential(*list(resnet.children())[:-1])
extractor.to(device).eval()

class MiniDS(Dataset):
    def __init__(self, paths): self.paths = paths
    def __len__(self): return len(self.paths)
    def __getitem__(self, i):
        try:
            im = cv2.imread(self.paths[i]); im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            im = cv2.resize(im, (224, 224))
            im = im / 255.0
            # Normalisation ImageNet stricte
            im = (im - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
            return torch.tensor(im.transpose(2,0,1), dtype=torch.float32)
        except: return torch.zeros((3,224,224))

loader = DataLoader(MiniDS(df_val_mini['path'].values), batch_size=32, shuffle=False)

feats = []
with torch.no_grad():
    for b in loader:
        b = b.to(device)
        f = extractor(b).squeeze(-1).squeeze(-1)
        feats.append(f.cpu().numpy())
X_mini = np.concatenate(feats)

# 3. PREDICTION
print("Test du modèle...")
model_m2 = xgb.XGBClassifier()
model_m2.load_model(path_mod)
y_pred = model_m2.predict(X_mini)

acc = accuracy_score(y_true, y_pred)

print(f"\n>>> VERDICT FINAL SUR 200 IMAGES : {acc:.4f} <<<")

if acc > 0.6:
    print("✅ SUCCÈS : Le modèle, l'encodeur et l'extracteur sont alignés.")
else:
    print("❌ ECHEC : Toujours une incohérence. Vérifie que l'entraînement Data Augmentation est bien TERMINÉ.")

--- VERIFICATION FINALE DE COMPATIBILITÉ (CORRIGÉE) ---
Chargement de l'encodeur (Vérifie la date du fichier !)...
Dernière modification du modèle : 2026-01-26 16:32:10.523593187
Extraction Features (Mode Torchvision)...
Test du modèle...

>>> VERDICT FINAL SUR 200 IMAGES : 0.9550 <<<
✅ SUCCÈS : Le modèle, l'encodeur et l'extracteur sont alignés.


In [9]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import timm
from torchvision import models
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score, classification_report
import xgboost as xgb
import joblib
import cv2
import torch.nn.functional as F
from tqdm import tqdm

print(">>> VERDICT FINAL : LE GRAND CONSEIL (VOTING) <<<")

# config
device = torch.device("cuda")
path_data = r"C:\Users\amisf\Desktop\datascientest_projet"
path_out = os.path.join(path_data, "implementation", "outputs")
num_classes = 27

# 1. Chargement Data Validation (La même que l'entrainement)
print("Chargement données...")
df = pd.read_csv(os.path.join(path_data, "data", "raw", "X_train_update.csv"))
y = pd.read_csv(os.path.join(path_data, "data", "raw", "Y_train_CVw08PX.csv"))
if 'prdtypecode' not in y.columns: y = y.rename(columns={y.columns[1]: 'prdtypecode'})
df = df.merge(y, left_index=True, right_index=True)

path_img = os.path.join(path_data, "data", "raw", "images", "images", "image_train")
if not os.path.exists(path_img): path_img = r"C:\Users\amisf\Desktop\datascientest_projet\data\raw\images\images\image_train"
df['path'] = df.apply(lambda r: os.path.join(path_img, f"image_{r['imageid']}_product_{r['productid']}.jpg"), axis=1)
df = df[df['path'].apply(os.path.exists)]

# Split et Encodage
le = preprocessing.LabelEncoder()
df['label'] = le.fit_transform(df['prdtypecode'])
_, val_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)
print(f"Validation sur {len(val_df)} images")

# 2. Classe Dataset Universelle
class UniversalDS(Dataset):
    def __init__(self, df, size, interpolation=cv2.INTER_LINEAR):
        self.paths = df['path'].values
        self.size = size
        self.inter = interpolation
    def __len__(self): return len(self.paths)
    def __getitem__(self, i):
        try:
            img = cv2.imread(self.paths[i]); img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (self.size, self.size), interpolation=self.inter)
            img = img / 255.0
            img = (img - np.array([0.485, 0.456, 0.406])) / np.array([0.229, 0.224, 0.225])
            return torch.tensor(img.transpose(2,0,1), dtype=torch.float32)
        except: return torch.zeros((3, self.size, self.size))

# 3. Calculs des Scores INDIVIDUELS

#  M1 : DINOv3 BEST 
print("\n1. M1: DINOv3 BEST...")
path_m1 = os.path.join(path_out, "model_DINOv3_BEST.pth")
if not os.path.exists(path_m1):
    # Fallback si le nom est different
    path_m1 = os.path.join(path_out, "M1_IMAGE_DeepLearning_DINOv3.pth")
    print(f"Note: Utilisation de {path_m1}")

mod_m1 = timm.create_model('vit_large_patch14_reg4_dinov2.lvd142m', pretrained=False, num_classes=num_classes)
mod_m1.load_state_dict(torch.load(path_m1))
mod_m1.to(device).eval()

loader_m1 = DataLoader(UniversalDS(val_df, 518, cv2.INTER_CUBIC), batch_size=16, shuffle=False) # 518px pour Dino
probs_m1 = []
with torch.no_grad():
    for b in tqdm(loader_m1, desc="M1 Inference"):
        b = b.to(device)
        with torch.amp.autocast('cuda'):
            probs_m1.append(F.softmax(mod_m1(b), dim=1).cpu().numpy())
p1 = np.concatenate(probs_m1)
del mod_m1; torch.cuda.empty_cache()

#  M3 : EfficientNet 
print("\n2. M3: EfficientNet...")
mod_m3 = models.efficientnet_b0(weights=None)
mod_m3.classifier[1] = nn.Linear(1280, num_classes)
mod_m3.load_state_dict(torch.load(os.path.join(path_out, "M3_IMAGE_Classic_EfficientNetB0.pth")))
mod_m3.to(device).eval()

loader_m3 = DataLoader(UniversalDS(val_df, 224), batch_size=64, shuffle=False)
probs_m3 = []
with torch.no_grad():
    for b in tqdm(loader_m3, desc="M3 Inference"):
        b = b.to(device)
        with torch.amp.autocast('cuda'):
            probs_m3.append(F.softmax(mod_m3(b), dim=1).cpu().numpy())
p3 = np.concatenate(probs_m3)
del mod_m3; torch.cuda.empty_cache()

#  M2 : XGBoost Champion 
print("\n3. M2: XGBoost Champion...")
# A. Alignement Labels
path_enc = os.path.join(path_out, "M2_IMAGE_XGBoost_Encoder.pkl")
le_xgb = joblib.load(path_enc)
y_true_aligned = le_xgb.transform(val_df['prdtypecode'])

# B. Features (Torchvision ResNet50)
print("Extraction Features (Strict)...")
resnet = models.resnet50(weights="IMAGENET1K_V1")
extractor = nn.Sequential(*list(resnet.children())[:-1])
extractor.to(device).eval()

loader_feat = DataLoader(UniversalDS(val_df, 224), batch_size=64, shuffle=False)
feats = []
with torch.no_grad():
    for b in tqdm(loader_feat, desc="M2 Extract"):
        b = b.to(device)
        f = extractor(b).squeeze(-1).squeeze(-1)
        feats.append(f.cpu().numpy())
x_val_xgb = np.concatenate(feats)

# C. Predict
model_m2 = xgb.XGBClassifier()
model_m2.load_model(os.path.join(path_out, "M2_IMAGE_Classic_XGBoost.json"))
p2 = model_m2.predict_proba(x_val_xgb)


# 4. FUSION STRATEGIQUE
print("\n>>> CALCUL DES SCORES <<<")

# Poids optimisés selon tes derniers entrainements
w1 = 2.0  # Dino (Solide)
w2 = 4.0  # XGBoost (Le Champion à 80%)
w3 = 1.0  # EffNet (Le soutien)

final_probs = (w1 * p1 + w2 * p2 + w3 * p3) / (w1 + w2 + w3)
y_pred_vote = final_probs.argmax(axis=1)

# Resultats individuels pour comparer
score_m1 = f1_score(y_true_aligned, p1.argmax(axis=1), average='weighted')
score_m2 = f1_score(y_true_aligned, p2.argmax(axis=1), average='weighted')
score_m3 = f1_score(y_true_aligned, p3.argmax(axis=1), average='weighted')
score_vote = f1_score(y_true_aligned, y_pred_vote, average='weighted')

print(f"M1 (Dino)    : {score_m1:.4f}")
print(f"M2 (XGBoost) : {score_m2:.4f}")
print(f"M3 (EffNet)  : {score_m3:.4f}")
print("-" * 30)
print(f"🏆 SCORE VOTING : {score_vote:.4f}")
print("-" * 30)

if score_vote > 0.83:
    print("C'est un score EXCELLENT pour ce projet.")

>>> VERDICT FINAL : LE GRAND CONSEIL (VOTING) <<<
Chargement données...
Validation sur 12738 images

1. M1: DINOv3 BEST...


M1 Inference: 100%|██████████| 797/797 [08:09<00:00,  1.63it/s]



2. M3: EfficientNet...


M3 Inference: 100%|██████████| 200/200 [01:32<00:00,  2.15it/s]



3. M2: XGBoost Champion...
Extraction Features (Strict)...


M2 Extract: 100%|██████████| 200/200 [01:30<00:00,  2.20it/s]



>>> CALCUL DES SCORES <<<
M1 (Dino)    : 0.7954
M2 (XGBoost) : 0.9269
M3 (EffNet)  : 0.6670
------------------------------
🏆 SCORE VOTING : 0.9272
------------------------------
C'est un score EXCELLENT pour ce projet.
